# 2 Player FPSB Auction with symmetric unfirorm valuation distributions

## Imports

In [ ]:
import os
import sys
root_path = os.path.abspath(os.path.join('..'))
if root_path not in sys.path:
    sys.path.append(root_path)

In [2]:
import torch
import torch.nn as nn
import torch.nn.utils as ut
from torch.optim.optimizer import Optimizer, required

In [3]:
from bnelearn.strategy import NeuralNetStrategy, TruthfulStrategy
from bnelearn.bidder import Bidder
from bnelearn.mechanism import FirstPriceSealedBidAuction, VickreyAuction
from bnelearn.optimizer import ES
from bnelearn.environment import Environment

In [4]:
from tensorboardX import SummaryWriter
import numpy as np
import matplotlib.pyplot as plt

In [5]:
torch.cuda.is_available()

True

## Settings

In [6]:
## Experiment setup
n_players = 2
n_items = 1
# valuation distribution
u_lo =0
u_hi =10

## Environment settings
#training batch size
batch_size = 2**10
input_length = 1

# model architecture
size_hidden_layer = 10

# optimization params
epoch = 20000
learning_rate = 2e-2
lr_decay = True
lr_decay_every = 2000
lr_decay_factor = 0.75

sigma = .1 #ES noise parameter
n_perturbations = 64

# plot and log training options
plot_epoch = 50
plot_points = min(150, batch_size)
sample_points = torch.from_numpy(np.linspace(u_lo, u_hi, u_hi+1)).float().view(-1, n_items).cuda()

# tensorboard writer settings

## Setting up the Environment

In [7]:
# for evaluation
def optimal_bid(valuation):
    return valuation / 2

In [8]:
# Wrapper transforming a strategy to bidder, used by the optimizer
def strat_to_bidder(strategy, batch_size):
    return Bidder.uniform(u_lo,u_hi, strategy, batch_size = batch_size, n_players=2)

In [9]:
model = NeuralNetStrategy(input_length, size_hidden_layer = size_hidden_layer, requires_grad=False).cuda()

In [10]:
mechanism = FirstPriceSealedBidAuction(cuda = True)
env = Environment(mechanism,
                  environment_agents = [], #dynamically built
                  max_env_size = 1, #
                  batch_size = batch_size,
                  n_players =2,
                  strategy_to_bidder_closure = strat_to_bidder
                 )
optimizer = ES(model=model, environment = env,
               lr = learning_rate, sigma=sigma, n_perturbations=n_perturbations)

In [11]:
def log_hyperparams(writer):
    writer.add_scalar('hyperparams/batch_size', batch_size)
    writer.add_scalar('hyperparams/size_hidden_layer', size_hidden_layer)
    writer.add_scalar('hyperparams/learning_rate', learning_rate)
    writer.add_scalar('hyperparams/sigma', sigma)
    writer.add_scalar('hyperparams/n_perturbations', n_perturbations)
    
    

## Training

In [ ]:
torch.cuda.empty_cache()
writer = SummaryWriter()
log_hyperparams(writer)

for e in range(epoch+1):    
    
    # lr decay?
    if lr_decay and e % lr_decay_every == 0 and e > 0:
        learning_rate = learning_rate * lr_decay_factor
        for param_group in optimizer.param_groups:
            param_group['lr'] = learning_rate
        writer.add_scalar('hyperparams/learning_rate', learning_rate, e)
        
    # always: do optimizer step
    utility = -optimizer.step()
    writer.add_scalar('eval/utility', utility, e) 
    
    # plot + eval
    if e % plot_epoch == 0:
        # plot current function output
        bidder = strat_to_bidder(model, batch_size)
        bidder.draw_valuations_()
        v = bidder.valuations
        b = bidder.get_action()
        share = (b/v).mean()
        writer.add_scalar('eval/utility', utility, e)
        writer.add_scalar('eval/share', share, e)       
            
        print("Epoch {}: \tavg bid: {:2f}, \tutility: {:2f}".format(e, share, utility))                
        
        # subsample points and plot
        v = v.detach().cpu().numpy()[:plot_points]
        b= b.detach().cpu().numpy()[:plot_points]
        optimal = optimal_bid(v)        
        fig = plt.figure()
        plt.plot(v,b, 'o', v, optimal, 'r-')        
        writer.add_figure('eval/bid_function', fig, e)        
        plt.show()
        
        # first step: write model
        if e==0:
            writer.add_graph(model, bidder.valuations)        
    
   
        
torch.cuda.empty_cache()
writer.close()

C:\Users\stefan\Anaconda3\envs\bnelearn\lib\site-packages\torch\nn\functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch 0: 	avg bid: 0.333420, 	utility: 4.193485
Epoch 50: 	avg bid: 1.180060, 	utility: 1.916827
Epoch 100: 	avg bid: 1.465558, 	utility: 0.951772
Epoch 150: 	avg bid: 1.899915, 	utility: 0.551881
Epoch 200: 	avg bid: 1.459402, 	utility: 0.221743
Epoch 250: 	avg bid: 1.911456, 	utility: 0.111375
Epoch 300: 	avg bid: 2.349579, 	utility: 0.124698
Epoch 350: 	avg bid: 1.921931, 	utility: 0.113362
Epoch 400: 	avg bid: 2.278556, 	utility: 0.022061
Epoch 450: 	avg bid: 1.738564, 	utility: 0.077999
Epoch 500: 	avg bid: 1.616777, 	utility: 0.229298
Epoch 550: 	avg bid: 1.608020, 	utility: 0.256441
Epoch 600: 	avg bid: 1.482962, 	utility: 0.149170
Epoch 650: 	avg bid: 1.561659, 	utility: 0.175611
Epoch 700: 	avg bid: 1.778017, 	utility: 0.313526
Epoch 750: 	avg bid: 1.443308, 	utility: 0.337184
Epoch 800: 	avg bid: 1.305993, 	utility: 0.313497
Epoch 850: 	avg bid: 1.303891, 	utility: 0.315209
Epoch 900: 	avg bid: 1.275763, 	utility: 0.461457
Epoch 950: 	avg bid: 1.224778, 	utility: 0.653687
Epo